In [23]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [1]:
import sys
import os

# Get the project root directory (parent of the notebooks folder)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the src folder to the Python path
sys.path.append(project_root)

In [2]:
import msgpack
import glob
from tqdm import tqdm
from PIL import Image
from io import BytesIO
import os 

#For Loading Images
from PIL import Image


import matplotlib.pyplot as plt
import os
import pandas as pd

from tqdm.notebook import trange, tqdm
from time import sleep

In [3]:
def get_image(record):
        return Image.open(BytesIO(record["image"]))

def images_read_save(shard_fnames):
    
    directory = "large-dataset-of-geotagged-images"
  
    # Parent Directory path 
    parent_dir = "../data/trial_data"

    # Path 
    path = os.path.join(parent_dir, directory) 
    os.mkdir(path) 
    print("Directory '% s' created" % directory) 
    coords = []
    image_id=[]
    for shard_fname in tqdm(shard_fnames):
        #print(f"Processing {shard_fname.split('/')[-1]}")
        with open(shard_fname, "rb") as infile:
            for record in msgpack.Unpacker(infile, raw=False):

                img = get_image(record)
                img.save('../data/trial_data/'+directory+'/'+str(record["latitude"])+'_'+str(record["longitude"])+'_'+''.join(record['id'].split('/')), 'JPEG')
                coords.append((record["latitude"], record["longitude"]))
                image_id.append('../data/trial_data/'+directory+'/'+str(record["latitude"])+'_'+str(record["longitude"])+'_'+''.join(record['id'].split('/')))

    return coords,image_id


In [4]:
dataset_dir = "../data/trial_data/shards/"
shard_fnames = [os.path.join(dataset_dir, fname) for fname in os.listdir(dataset_dir) if fname.endswith(".msg")]
coords, image_id=images_read_save(shard_fnames)

def create_df(image_id,coords):
    df_1=pd.DataFrame(image_id)
    df_1.rename(columns={df_1.columns[0]:'id'},inplace=True)
    df_2=pd.DataFrame(coords)
    df_2.rename(columns={df_2.columns[0]:'lat',df_2.columns[1]:'lng'},inplace=True)
    df_final=pd.concat([df_1,df_2],axis=1)
    return df_final
    
df=create_df(image_id, coords)

df.head()



Directory 'large-dataset-of-geotagged-images' created


  0%|          | 0/1 [00:00<?, ?it/s]

,id,lat,lng
0,../data/trial_data/large-dataset-of-geotagged-...,-34.574164,-58.453917
1,../data/trial_data/large-dataset-of-geotagged-...,47.608571,-122.340247
2,../data/trial_data/large-dataset-of-geotagged-...,43.266050,-79.904980
3,../data/trial_data/large-dataset-of-geotagged-...,42.267385,-71.807584
4,../data/trial_data/large-dataset-of-geotagged-...,57.040974,8.495188


In [9]:
import reverse_geocoder as rg
import json

coordinates = list(zip(df['lat'], df['lng']))

# Use tqdm to show progress
tqdm.pandas()

# Reverse geocode using mode=2 (batch processing)
results = rg.RGeocoder(mode=2).query(coordinates)

country_code_to_label = json.load(open("../data/trial_data/country_code_to_index.json", "r"))
# Add country_code to DataFrame from results
df["label"] = [country_code_to_label[result['cc']] for result in results]

# Display the DataFrame with the new column
data = list(zip(df['id'], df['label']))
print(data)

[('../data/trial_data/large-dataset-of-geotagged-images/-34.574164_-58.453917_75553923967888.jpg', 2), ('../data/trial_data/large-dataset-of-geotagged-images/47.608571_-122.340247_f2b22703239886.jpg', 130), ('../data/trial_data/large-dataset-of-geotagged-images/43.26605_-79.90498_e5d62090373280.jpg', 20), ('../data/trial_data/large-dataset-of-geotagged-images/42.267385_-71.807584_8acd148928726.jpg', 130), ('../data/trial_data/large-dataset-of-geotagged-images/57.040974_8.495188_50da5195081755.jpg', 30), ('../data/trial_data/large-dataset-of-geotagged-images/13.7455_100.533166_c6ee9310838382.jpg', 121), ('../data/trial_data/large-dataset-of-geotagged-images/-33.042198_-71.605625_d4176667367483.jpg', 21), ('../data/trial_data/large-dataset-of-geotagged-images/41.161734_-112.019659_9b037893100200.jpg', 130), ('../data/trial_data/large-dataset-of-geotagged-images/35.68164_139.764375_abcf3349427276.jpg', 58), ('../data/trial_data/large-dataset-of-geotagged-images/45.182127_0.723252_a9546582